In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git torch accelerate pandas

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
from huggingface_hub import login

hf_token = "YOUR_HUGGING_FACE_TOKEN_HERE"
login(token=hf_token)

In [ ]:
model_id = "bigcode/starcoder2-3b"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=hf_token,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
TEMPLATE = """
You are an expert CS instructor. Analyze the following Python student submission.

Code:
{student_code}

Tasks:
- Summarize in 2 sentences what the code does.
- List 2–4 core CS concepts the student must know.
- Identify one likely misconception or reasoning gap.
- Provide a single-sentence hint to fix the issue (do NOT write runnable code).
- Give a competence score (1–5) with one-line justification.

Respond **ONLY in JSON format** like this (do NOT include extra text, examples, or instructions):
{{
  "summary": "...",
  "concepts": ["...", "..."],
  "misconception": "...",
  "hint": "...",
  "score": ...,
  "justification": "..."
}}
"""

In [ ]:
student_codes = [
    "def add(a,b): return a+b",
    "for i in range(1,10): print(i*2)",
    "x = [i**2 for i in range(5)]"
]

In [ ]:
from transformers import pipeline
import re
import json
import pandas as pd


In [ ]:
results = []

for student_code in student_codes:
    prompt = TEMPLATE.format(student_code=student_code)

    out = pipe(prompt, max_new_tokens=300, do_sample=False, return_full_text=False)
    generated_text = out[0]["generated_text"].strip()

    # Extract JSON only
    match = re.search(r"\{.*\}", generated_text, re.DOTALL)
    if match:
        try:
            analysis_json = json.loads(match.group(0))
        except json.JSONDecodeError:
            analysis_json = {}
    else:
        analysis_json = {}

    results.append({
        "student_code": student_code,
        "summary": analysis_json.get("summary", ""),
        "concepts": ", ".join(analysis_json.get("concepts", [])),
        "misconception": analysis_json.get("misconception", ""),
        "hint": analysis_json.get("hint", ""),
        "score": analysis_json.get("score", ""),
        "justification": analysis_json.get("justification", "")
    })

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
df = pd.DataFrame(results)
df.to_csv("student_analysis.csv", index=False)
df.head()

,student_code,summary,concepts,misconception,hint,score,justification
0,"def add(a,b): return a+b",This code adds two numbers together.,"..., ...",The student is not sure what the code does.,The code adds two numbers together.,3,The student is not sure what the code does.
1,"for i in range(1,10): print(i*2)","Prints the numbers 2, 4, 6, 8, 10, 12, 14, 16, 18","for loops, range",The student thinks the code prints the numbers...,The student should use the range function to p...,3,The student knows that the range function retu...
2,x = [i**2 for i in range(5)],This code creates a list of the squares of the...,"list comprehension, for loop",The student thinks the code creates a list of ...,The student should use the range function to c...,3,The student knows that the range function crea...
